# Arbeidsark: Multivariat Dataanalyse i Python

## Målsetninger:
- Laste inn et datasett fra en CSV-fil.
- Utføre multivariat regresjonsanalyse ved hjelp av Ordinære Minste Kvadraters (OLS) metode.
- Teste forutsetningene for OLS-regresjon:
  - Linearitet
  - Homoskedastisitet
  - Normalitet av residualer
  - Multikollinearitet
- Bruke dummyvariabler i regresjonsmodeller.
- Tolke R-square, justert R-squared, koeffisienter og signifikanstester.
- Visualisere nøkkelresultater.

## Oppgaver for modulen 
- Last inn datasettet og undersøk strukturen ved hjelp av .head() og .info().
- Utfør OLS-regresjon ved å bruke de oppgitte variablene.
- Tolk resultatene: Diskuter R-kvadrat, justert R-kvadrat, koeffisienter og p-verdier.
- Test forutsetningene for OLS-regresjon og rapporter funnene.
- Arbeid med dummyvariabler: Inkluder en kategorisk variabel i regresjonsmodellen og tolk koeffisientene.
- Visualiser resultatene for å støtte analysen din.

### Aller første du må gjøre, er å importere biblioteker som skal brukes

In [ ]:
# Steg 1: Importer nødvendige biblioteker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor



## 1. Last inn datasettet
(Husk å kjør koden i data-cellene for å få resultater. Dette gjøres ved å sette markøren i cellen, og enten klikke [ctrl]+[enter] på tastaturet, eller klikke "play" knappen på menyen over.

In [ ]:
# Last inn datasettet fra en CSV-fil
# Bytt ut 'your_file.csv' med filstien til datasettet ditt
file_path = 'datafil'
data = pd.read_csv(file_path)

# Vis de første radene i datasettet
data.head()

## 1b: Still noen forskningsspørsmål
Se på variablene, og lag noen aktuelle forsknignsspørsmål. Husk på kravene til gode spørsmål. 
Et slik kan være: "Hvordan påvirker kjønn, alder og utdanning grunnlønnen til ansatte i dette firmaet?" 

### skriv inn spørsmål her: 

......


## 2. Utfør OLS-regresjon
OLS brukes for å estimere påvirkningen av uavhengig variabler på en avhengig variable, den vi er interessert i forstå.

### oppgave:
- Utfør OLS-regresjon ved å bruke et utvalg av variabler som svarer på spørsmålet du har stilt over.
- forsøk å still et spørsmål som inkluderer kjønn. (Hint: du må gjøre noe med variablene for å kunne bruke den i OLS regresjon)


In [ ]:
# Definer avhengige og uavhengige variabler
# Bytt ut 'dependent_var' med navnet på din avhengige variabel
# Bytt ut 'independent_var1', 'independent_var2', osv. med navnene på dine uavhengige variabler
dependent_var = 'variabel'
independent_vars = ['variabel', 'variabel']  # Legg til flere variabler om nødvendig

# Legg til en konstant for intercept
X = sm.add_constant(data[independent_vars])
y = data[dependent_var]

# Utfør OLS-regresjonsanalyse
model = sm.OLS(y, X).fit()

# Skriv ut sammendraget av regresjonsmodellen
print(model.summary())


## 3. Tolke Resultatene
R-kvadrat: Prosentandel av variansen forklart av modellen.
Justerte R-kvadrat: Justert for antall prediktorer.
Koeffisienter: Viser effekten av hver uavhengig variabel på den avhengige variabelen.
P-verdier: Tester nullhypotesen om at koeffisienten er null.
### oppgave:
Forklar hva hver av disse vil si, gitt modellen over.

## 4. Test Forutsetningene for Regressjonsanalyse
- a) Lineæritet
- b) Normalitet av Residualer
- c) Homoskedastisitet
- d) Multikollinearitetet

In [ ]:
# Visualiser sammenhengen
# a) Lineæritet
sns.pairplot(data, x_vars=independent_vars, y_vars=dependent_var, kind='reg')
plt.show()
# Forslag til tolkning etter visualisering
print("Tolk plottet:")
print("- Se etter positive eller negative sammenhenger mellom variablene.")
print("- Undersøk om punktene klustrer seg rundt regresjonslinjen for å vurdere styrken på sammenhengen.")
print("- Identifiser uteliggere eller ikke-lineære mønstre.")
print("- Vurder spredningen av punktene for å evaluere konsistensen i variansen.")



In [ ]:
# b) Normalitet av Residualer
# Plot histogram og Q-Q-plott av residualer
#residuals = model.resid

standardized_residuals = (residuals - residuals.mean()) / residuals.std()
sm.qqplot(standardized_residuals, line='45')
plt.title('Q-Q Plot of Standardized Residuals')
plt.show()

print("Tolkning av Q-Q-plott:")
print("- Hvis punktene følger den diagonale linjen tett, er residualene tilnærmet normalfordelt.")
print("- Avvik fra linjen, spesielt i ytterkantene, kan indikere avvik fra normalfordeling.")

#Shapiro Wilks
from scipy.stats import shapiro

# Perform Shapiro-Wilk test
shapiro_stat, shapiro_p = shapiro(standardized_residuals)

print(f"Shapiro-Wilk Test Statistic: {shapiro_stat}")
print(f"P-value: {shapiro_p}")

# Tolkning av Shapiro-Wilk-test
if shapiro_p < 0.05:
    print("Residualene er ikke normalfordelt (p < 0.05). Dette kan påvirke gyldigheten av enkelte regresjonsanalyser.")
else:
    print("Residualene er normalfordelt (p ≥ 0.05). Dette støtter en av forutsetningene for OLS-regresjon.")



In [ ]:
# c) Homoskedastisitet
# Utfør Breusch-Pagan-test for homoskedastisitet
_, pval, _, _ = het_breuschpagan(model.resid, model.model.exog)
print(f"P-verdi for homoskedastisitet: {pval}")
if pval < 0.05:
    print("Modellen har heteroskedastisitet (p < 0.05).")
else:
    print("Modellen har ikke heteroskedastisitet (p ≥ 0.05).")


In [ ]:
# d) Multikollinearitet
# Beregn Variance Inflation Factor (VIF) for hver prediktor
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factor (VIF):")
print(vif_data)
# Tolkning av VIF
print("\nTolkning av VIF:")
print("- En VIF-verdi under 5 indikerer lav multikollinearitet, noe som er akseptabelt.")
print("- En VIF-verdi mellom 5 og 10 kan indikere moderat multikollinearitet.")
print("- En VIF-verdi over 10 indikerer høy multikollinearitet, og variabelen kan forårsake problemer i regresjonsmodellen.")

## 5. Bruk av Dummyvariabler
For å analysere kategoriske variabler i en regresjonsmodell, må de konverteres til numerisk. Dette gjøres ved å bruke Dummy variabler, der det er en mindre dummy-variable enn alternative kategorier. (Slå opp mer om dummyvariabler om du er usikker)

In [ ]:
# Opprett dummyvariabler for en kategorisk variabel
# Bytt ut 'categorical_var' med navnet på din kategoriske variabel

# Create dummy variables for 'edu'
edu_dummies = pd.get_dummies(data['variabel'], prefix='det du vil ha forran verdien i variabelen')

# Convert True/False to 1/0
edu_dummies = edu_dummies.astype(int)  

# Concatenate dummy variables to the original DataFrame
data = pd.concat([data, edu_dummies], axis=1)

# Vis de første radene i datasettet
data.head()



## 5b. OLS med dummyvariabler
Kjør ny regresjon med dummyvariabler

In [ ]:
# Definer avhengige og uavhengige variabler
# Bytt ut 'dependent_var' med navnet på din avhengige variabel
# Bytt ut 'independent_var1', 'independent_var2', osv. med navnene på dine uavhengige variabler
dependent_var = 'variabel'
independent_vars = ['variabel', 'variabel', 'dummy_variabel_1', 'dummy_variabel_2', 'så mange fler du trenger']  # Legg til flere variabler om nødvendig

# Legg til en konstant for intercept
X = sm.add_constant(data[independent_vars])
y = data[dependent_var]

# Utfør OLS-regresjonsanalyse
model = sm.OLS(y, X).fit()

# Skriv ut sammendraget av regresjonsmodellen
print(model.summary())

## 6. Visualiser Nøkkelresultater
- Regresjonslinje for en Enkeltprediktor
- Koeffisientplot

In [ ]:
# Visualiser regresjonslinjen for en uavhengig variabel
# Bytt ut 'independent_var1' med navnet på prediktoren
sns.regplot(x='independent_var1', y=dependent_var, data=data)
plt.title(f"Regresjonslinje: {dependent_var} vs perfEval")
plt.xlabel('independent_var1')
plt.ylabel(dependent_var)
plt.show()



In [ ]:
# Plot koeffisienter og konfidensintervaller
coefficients = model.params
conf_int = model.conf_int()
plt.figure(figsize=(8, 6))
plt.errorbar(coefficients.index, coefficients, 
             yerr=[coefficients - conf_int[0], conf_int[1] - coefficients], fmt='o')
plt.axhline(0, color='red', linestyle='--')
plt.title("Koeffisienter og Konfidensintervaller")
plt.xticks(rotation=45)
plt.show()



## 5. Drøft innsikter
I analysene over, bør du ha fått noen innsikter fra analysene, og tolk visualiseringene. Presenter disse her: 
(Hint: hva ser du om sammenheng mellom forskjellige typer variabler: kategoriske, numeriske, m.m. om variablene du så på?)
Sørg for at du har svart på spørsmålene du stilte øverst. 

### Skriv svaret her
...